### Load required libraries

In [3]:
import pandas as pd
import datetime as dt
import numpy as np

### Correction on Method used to select last month in Period column

In [28]:
def anomaly_func(file):
  """
  This function identifies stores who recorded no sales and closing stock in the current month, ranked according to a given threshold
  """
  #Read Excel file
  df = pd.read_excel(file)

  periodsort = sorted(df['Period'].unique()) # arrange date periods and select the current month
  most_recent_year_month_df = df[df['Period']==periodsort[-1]]

  # Create 'anomaly' column in 'most_recent_year_month_df' 
  most_recent_year_month_df['Anomaly'] = most_recent_year_month_df.apply(lambda x: 1 if x['CLOSING STOCK'] == 0 and x['SALES'] == 0 else 0, axis = 1)

  # Subset and create dataframe to select anomalies that are of value 1
  anomaly_df = most_recent_year_month_df[most_recent_year_month_df['Anomaly']==1]

  # New Dataframe to Count flagged items per store (for last month)
  anomaly_products_count = anomaly_df.groupby(['Store Name', 'Store Code','Auditors Name'])['Anomaly'].sum()
  count_df = anomaly_products_count.reset_index()
  count_df = count_df.rename(columns ={'Anomaly': 'Flagged items per store'})

  # Save names of anomaly store owners
  anomaly_store_owners = anomaly_df['Store Name'].unique().tolist()

  # New Dataframe to Count total items per store for store owners with anomalies (for last month)
  products_SKU = most_recent_year_month_df[most_recent_year_month_df['Store Name'].isin(anomaly_store_owners)]
  month_total_product_count = products_SKU.groupby(['Store Name', 'Store Code','Auditors Name'])['Anomaly'].count()
  sku_df = month_total_product_count.reset_index()
  sku_df = sku_df.rename(columns ={'Anomaly': 'Sum of items per store'})

  # Dataframe in expected format
  final_df = count_df.merge(sku_df, on=['Store Name', 'Store Code', 'Auditors Name'], suffixes=('_x', '_y'))
  final_df['Anomaly items per store / %'] = round(final_df['Flagged items per store']/ final_df['Sum of items per store'] * 100, 2)

  # Further sorting based on expected results
  global output_file
  output_file = final_df[final_df['Anomaly items per store / %']>20] # anomalies higher than 20%

  # # Create an ExcelWriter object
  # writer = pd.ExcelWriter("programming-assessment.xlsx")

  # # Save the DataFrame to a worksheet in the Excel file
  # output_file.to_excel(writer, sheet_name="anomaly-detection")

  # # Save the ExcelWriter object
  # writer.save()
  # Save dataframe to excel
  #output_file.to_excel('flagged-items-count.xlsx', index=False)

  
  return output_file#print(output_file, output_file['Count SKU'].sum(), sep='\n\n\n')

anomaly_func(r"C:\Users\Akosua\Documents\Andrew\Interview Assessment Redo\DS Maverick Sample Dataset.xlsx")

C:\Users\Akosua\AppData\Local\Temp\ipykernel_17208\3487283827.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_recent_year_month_df['Anomaly'] = most_recent_year_month_df.apply(lambda x: 1 if x['CLOSING STOCK'] == 0 and x['SALES'] == 0 else 0, axis = 1)


,Store Name,Store Code,Auditors Name,Flagged items per store,Sum of items per store,Anomaly items per store / %
5,ADOM GROCERY,GH0002136103,ALIMA ALIU,19,85,22.35
6,ADOM SHOPPING CENTER,GH0005127127,LEWIS AGBOW,51,108,47.22
11,AUNTY ANITA TAB,GH0002136179,ALIMA ALIU,10,40,25.00
20,BY HIS GRACE(EVELYN BOADI),GH000312566,RICHMOND AWUNI,19,56,33.93
60,JUMADA ENT,GH000512722,LEWIS AGBOW,11,53,20.75
61,KAYITE WALE,GH000413021,ADAM JACOB,68,235,28.94
67,MABEL ENT.,GH0005127111,LEWIS AGBOW,29,108,26.85
69,MAMA*S MART,GH0004133102,ADAM JACOB,38,126,30.16
74,MMA FATI PROVISION,GH000413314,ADAM JACOB,30,112,26.79
79,NKOSOA TAB,GH0002136194,ALIMA ALIU,6,27,22.22


### Attempt at Creating Classes of Functions

In [8]:
class AnomalyStores:

    def __init__(self, file_path):
        self.file_path = file_path

    def anomaly_func(self):
      """
      This function identifies stores who recorded no sales and closing stock in the current month, ranked according to a given threshold
      """
      df = pd.read_excel(self.file_path) # Read Excel file

      periodsort = sorted(df['Period'].unique()) # arrange date periods and select the current month
      most_recent_year_month_df = df[df['Period']==periodsort[-1]]

      # Create 'anomaly' column in 'most_recent_year_month_df' 
      most_recent_year_month_df['Anomaly'] = most_recent_year_month_df.apply(lambda x: 1 if x['CLOSING STOCK'] == 0 and x['SALES'] == 0 else 0, axis = 1)

      # Subset and create dataframe to select anomalies that are of value 1
      anomaly_df = most_recent_year_month_df[most_recent_year_month_df['Anomaly']==1]

      # New Dataframe to Count flagged items per store (for last month)
      anomaly_products_count = anomaly_df.groupby(['Store Name', 'Store Code','Auditors Name'])['Anomaly'].sum()
      count_df = anomaly_products_count.reset_index()
      count_df = count_df.rename(columns ={'Anomaly': 'Flagged items per store'})

      # Save names of anomaly store owners
      anomaly_store_owners = anomaly_df['Store Name'].unique().tolist()

      # New Dataframe to Count total items per store for store owners with anomalies (for last month)
      products_SKU = most_recent_year_month_df[most_recent_year_month_df['Store Name'].isin(anomaly_store_owners)]
      month_total_product_count = products_SKU.groupby(['Store Name', 'Store Code','Auditors Name'])['Anomaly'].count()
      sku_df = month_total_product_count.reset_index()
      sku_df = sku_df.rename(columns ={'Anomaly': 'Sum of items per store'})

      # Dataframe in expected format
      final_df = count_df.merge(sku_df, on=['Store Name', 'Store Code', 'Auditors Name'], suffixes=('_x', '_y'))
      final_df['Anomaly items per store / %'] = round(final_df['Flagged items per store']/ final_df['Sum of items per store'] * 100, 2)

      # Further sorting based on expected results
      global output_file
      output_file = final_df[final_df['Anomaly items per store / %']>20] # anomalies higher than 20%
      
      return output_file


In [9]:
#Create an instance of the SalesAnomalyDetector class by passing the file path of the Excel file as an argument. For example:
detector = AnomalyStores(file_path=r'C:\Users\Akosua\Documents\Andrew\Interview Assessment Redo\DS Maverick Sample Dataset.xlsx')

#Call the detect_anomalies() method on the instance. This will return a DataFrame containing the stores with sales anomalies.
anomalies_df = detector.anomaly_func()

#Print the DataFrame to see the results.
print(anomalies_df)


                                        Store Name     Store Code  \
5                                     ADOM GROCERY   GH0002136103   
6                             ADOM SHOPPING CENTER   GH0005127127   
11                                 AUNTY ANITA TAB   GH0002136179   
20                      BY HIS GRACE(EVELYN BOADI)    GH000312566   
60                                      JUMADA ENT    GH000512722   
61                                     KAYITE WALE    GH000413021   
67                                      MABEL ENT.   GH0005127111   
69                                     MAMA*S MART   GH0004133102   
74                              MMA FATI PROVISION    GH000413314   
79                                      NKOSOA TAB   GH0002136194   
90                                 NO NAME - KELLY   GH0005127207   
91                                 NO NAME - MAVIS   GH0005127142   
95                                  NO NAME - VERO  GH00051271710   
108                               

C:\Users\Akosua\AppData\Local\Temp\ipykernel_17664\2917472515.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  most_recent_year_month_df['Anomaly'] = most_recent_year_month_df.apply(lambda x: 1 if x['CLOSING STOCK'] == 0 and x['SALES'] == 0 else 0, axis = 1)


In [10]:
class AnomalyItemHistory:

    def __init__(self, file_path):
        self.file_path = file_path

    def reference_check(self):
        """
        This function identifies the flagged items and traces back the history of these items over the past
        3 months
        """

        df = pd.read_excel(self.file_path) # Read Excel file

        # Filter dataframe by date period of last month
        df.sort_values(by='Period', ascending=True, inplace=True) # sort by date ascending order
        end_date = df.iloc[-1,0] # select the most recent date
        start_date = end_date - pd.DateOffset(months=0) # set the starting date
        filtered_df = df.loc[(df['Period'] >= start_date) & (df['Period'] <= end_date)] # filter dataframe for last month

        # Call names of flagged stores and filter date period dataframe

        global output_file # flagged stores called from anomaly_func
        suspicious_stores = output_file['Store Code'].unique().tolist() # save store names as a list
        extended_months = filtered_df[filtered_df['Store Code'].isin(suspicious_stores)] # filter last month data by flagged store names

        # Apply flag settings to dataframe
        extended_months['Anomaly'] = extended_months.apply(lambda x: 1 if x['CLOSING STOCK'] == 0 and x['SALES'] == 0 else 0, axis = 1)

        # Subset dataframe to select anomalies that are of value 1
        extended_anomaly_df = extended_months[extended_months['Anomaly']==1]

        # Filter dataframe by date period ranging 3 months behind the last (recent) date
        start_period = end_date - pd.DateOffset(months=2) # set start period 
        df_for_3_months = df.loc[(df['Period'] >= start_period) & (df['Period'] <= end_date)] # filter dataframe for last 3 months   

        # Use "df_for_3_months" and "extended_anomaly_df" which represent data for last 3 months and data for last month respectively
        # Select the column names of interest
        column_names = ["Store Code", "Item Code"]

        # Create a new DataFrame using last month's records with the columns of interest
        df_subset = extended_anomaly_df[column_names]

        subfinal = df_for_3_months.merge(df_subset, on= column_names) # merge both dataframes on common columns, found in variable

        # Drop undesired columns
        subfinal.drop(columns=['Country','City','Previous visit date', 'Current visit date',
        'Barcode', 'SEGMENT', 'MANUFACTURER',
            'BRAND NAME', 'WEIGHT', 'Item_Unit', 'Packaging',
        'OPENING STOCK', 'PRICE'], inplace= True)
        
        # Arrange desired record in pivot table
        table2 = pd.pivot_table(subfinal, values= ['CLOSING STOCK', 'TOTAL PURCHASE', 'SALES'],
                            index= ['Store Name', 'Store Code', 'Auditors Name', 'Item Code', 'ITEM NAME', 'Channel', 'CATEGORY'],
                            columns= ['Period'])

        return subfinal



In [11]:
#Create an instance of the SalesAnomalyDetector class by passing the file path of the Excel file as an argument. For example:
detector = AnomalyItemHistory(file_path=r'C:\Users\Akosua\Documents\Andrew\Interview Assessment Redo\DS Maverick Sample Dataset.xlsx')

#Call the detect_anomalies() method on the instance. This will return a DataFrame containing the stores with sales anomalies.
anomalies_df = detector.reference_check()

#Print the DataFrame to see the results.
print(anomalies_df)


         Period      Channel    Store Code           Store Name Auditors Name  \
0    2021-01-01  Open_Market  GH0004133165  SUHUYINI ENTERPRISE    ADAM JACOB   
1    2021-02-01  Open_Market  GH0004133165  SUHUYINI ENTERPRISE    ADAM JACOB   
2    2021-03-01  Open_Market  GH0004133165  SUHUYINI ENTERPRISE    ADAM JACOB   
3    2021-01-01  Open_Market  GH0004133165  SUHUYINI ENTERPRISE    ADAM JACOB   
4    2021-02-01  Open_Market  GH0004133165  SUHUYINI ENTERPRISE    ADAM JACOB   
...         ...          ...           ...                  ...           ...   
2125 2021-03-01  Mini_Stores  GH0002136182           SISTER YAA    ALIMA ALIU   
2126 2021-02-01    Groceries  GH0002136103         ADOM GROCERY    ALIMA ALIU   
2127 2021-03-01    Groceries  GH0002136103         ADOM GROCERY    ALIMA ALIU   
2128 2021-02-01    Groceries  GH0002136103         ADOM GROCERY    ALIMA ALIU   
2129 2021-03-01    Groceries  GH0002136103         ADOM GROCERY    ALIMA ALIU   

               CATEGORY    

C:\Users\Akosua\AppData\Local\Temp\ipykernel_17664\1593191155.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extended_months['Anomaly'] = extended_months.apply(lambda x: 1 if x['CLOSING STOCK'] == 0 and x['SALES'] == 0 else 0, axis = 1)


In [11]:
class SalesAnomalyDetector:

    def __init__(self, file_path):
        self.file_path = file_path

    def detect_anomalies(self):
        """
        This function compares the sales value recorded by shops in the present month against the past
        4 months and determines whether the present sales value has defects
        """
        df = pd.read_excel(self.file_path) # Read Excel file
        
        sales_df = df[["Period", "Store Code", "Store Name", "Auditors Name", "SALES", "PRICE"]] # Subset relevant columns and create dataframe 

        sales_df['Non-zero-sales'] = sales_df['SALES'].abs() # Compute non-zero sales 

        sales_df['Sales Value'] = sales_df['Non-zero-sales']* sales_df['PRICE'] # Compute sales value column

        subset = sales_df.groupby(["Period", "Store Code", "Store Name", "Auditors Name"])['Sales Value'].sum().reset_index(name='Sales Value') # groupby

        # Arrange desired record in pivot table
        table = pd.pivot_table(subset, values= ['Sales Value'],
                           index= ["Store Code", "Store Name", "Auditors Name"],
                           columns= ['Period'])
        
        new_sales = table.reset_index()#.stack() # change shape of pivot table

        # Retrieve Month-on-Month sales values for each of the past 4 months and the current month
        df_list = []
        months_weeks = [16, 12, 8, 4, 0] # **NB. needs to be changed based on the current month**
        num = iter(list(range(4,new_sales.shape[1])))
        part_newdf = new_sales.iloc[:, :3]  # subset indexed pivot table to pick out Store Code, Store Name and Auditor's Name
        part_newdf.columns = part_newdf.columns.droplevel(level=1) # remove the period index  
        for week in months_weeks:
            td = dt.timedelta(weeks=week)
            month_sales = sales_df.iloc[-1, 0] - td
            sample_df = pd.DataFrame({f'Sales value {month_sales.month_name()}': new_sales.loc[:, new_sales.columns[next(num)]]})
            df_list.append(sample_df)
        concatenated_df = pd.concat(df_list, axis=1)
        part_newdf = part_newdf.join(concatenated_df)

        # Compute mean of Sales value per store for the past 4 months
        past_4_months_df = part_newdf.iloc[:, 3:7]  #  Selects the past 4 month sales averages
        row_average = round(past_4_months_df.mean(axis=1), 2) # Average sales value for the month periods
        row_average_df = pd.DataFrame({'Avg Sales value past 4 months': row_average}) # create dataframe of series object 'row_average'
        part_newdf = part_newdf.join(row_average_df) # add row_average df to the dataframe

        # Filter for stores based on thresholds
        threshold_value = 0.3
        part_newdf['Lower Threshold past 4 months'] = round(part_newdf['Avg Sales value past 4 months']* (1-threshold_value), 2)
        part_newdf['Upper Threshold past 4 months'] = round(part_newdf['Avg Sales value past 4 months']* (1+threshold_value), 2)

        
        # Filter and Flag
        # Create boolean masks for each condition
        within_limit = part_newdf['Sales value March'].between(part_newdf['Lower Threshold past 4 months'], part_newdf['Upper Threshold past 4 months'])
        upper_breach = part_newdf['Sales value March'] > part_newdf['Upper Threshold past 4 months']
        lower_breach = part_newdf['Sales value March'] < part_newdf['Lower Threshold past 4 months']

        # Define the corresponding flags for each condition
        conditions = [within_limit, upper_breach, lower_breach]
        choices = ['Within limit', 'Upper threshold breached', 'Lower threshold breached']

        # Use numpy.select to assign the corresponding flag to each row
        part_newdf['Sales Value Status'] = np.select(conditions, choices, default='Unknown')

        # Store values that breach limits
        full_df = part_newdf[~(part_newdf['Sales Value Status']=='Within limit')]

        return part_newdf[~(part_newdf['Sales Value Status']=='Within limit')]

In [13]:
#Create an instance of the SalesAnomalyDetector class by passing the file path of the Excel file as an argument. For example:
detector = SalesAnomalyDetector(file_path=r'C:\Users\Akosua\Documents\Andrew\Interview Assessment Redo\DS Maverick Sample Dataset.xlsx')

#Call the detect_anomalies() method on the instance. This will return a DataFrame containing the stores with sales anomalies.
anomalies_df = detector.detect_anomalies()

#Print the DataFrame to see the results.
print(anomalies_df)

       Store Code              Store Name       Auditors Name  \
1     GH000111433         NO NAME CHRISTY        LISA BOATENG   
3     GH000111456          JOLLO SHOPPING        LISA BOATENG   
4     GH000111466       STELLA ENTERPRISE        LISA BOATENG   
5     GH000111474         EMMA ENTERPRISE        LISA BOATENG   
6     GH000111489           NO NAME DORIS        LISA BOATENG   
..            ...                     ...                 ...   
199   GH000512745  CEE LOVELY SUPERMARKET         LEWIS AGBOW   
200   GH000512761                PSALM 21         LEWIS AGBOW   
202  GH0005127810           AMENUVEVE ENT         LEWIS AGBOW   
203   GH000512797            ADINKRA ENT.         LEWIS AGBOW   
217   GH000514424            NYAME NA AYE  NANA KOFI AMOABENG   

     Sales value November  Sales value December  Sales value January  \
1                 1591.90               1551.00              3207.70   
3                  815.90               1517.70              2452.70   
4  

C:\Users\Akosua\AppData\Local\Temp\ipykernel_17208\3362384837.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Non-zero-sales'] = sales_df['SALES'].abs() # Compute non-zero sales
C:\Users\Akosua\AppData\Local\Temp\ipykernel_17208\3362384837.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Sales Value'] = sales_df['Non-zero-sales']* sales_df['PRICE'] # Compute sales value column


In [15]:
class ItemAnomalyDetector:

    def __init__(self, file_path):
        self.file_path = file_path

    def item_sales_anomaly_func(self):
        """
        This function compares the sales value recorded by shops in the present month against the past
        4 months and determines whether the present sales value has defects
        """
        df = pd.read_excel(self.file_path) # Read Excel file
        
        # Subset and create dataframe with relevant columns to sales analysis
        sales_df = df[["Period", "Item Code", "ITEM NAME", "CATEGORY", "Auditors Name", "SALES", "PRICE"]]

        sales_df['Non-zero-sales'] = sales_df['SALES'].abs() # Compute non-zero sales 

        sales_df['Sales Value'] = sales_df['Non-zero-sales']* sales_df['PRICE'] # Compute sales value column

        subset = sales_df.groupby(["Period", "Item Code", "ITEM NAME", "CATEGORY", "Auditors Name"])['Sales Value'].sum().reset_index(name='Sales Value')

        # Arrange desired record in pivot table
        table = pd.pivot_table(subset, values= ['Sales Value'],
                            index= ["Item Code", "ITEM NAME", "CATEGORY", "Auditors Name"],
                            columns= ['Period'])
        
        new_sales = table.reset_index()#.stack()

        # Retrieve Month-on-Month sales values for each of the past 4 months and the current month
        df_list = []
        months_weeks = [16, 12, 8, 4, 0]
        num = iter(list(range(4,new_sales.shape[1])))
        part_newdf = new_sales.iloc[:, :3]  # subset indexed pivot table to pick out Store Code, Store Name and Auditor's Name
        part_newdf.columns = part_newdf.columns.droplevel(level=1) # remove the period index  
        for week in months_weeks:
            td = dt.timedelta(weeks=week)
            month_sales = sales_df.iloc[-1, 0] - td
            sample_df = pd.DataFrame({f'Sales value {month_sales.month_name()}': new_sales.loc[:, new_sales.columns[next(num)]]})
            df_list.append(sample_df)
        concatenated_df = pd.concat(df_list, axis=1)
        part_newdf = part_newdf.join(concatenated_df)

        # Compute mean of Sales value per store for the past 4 months
        past_4_months_df = part_newdf.iloc[:, 4:8]  #  Selects the past 4 month sales averages
        row_average = round(past_4_months_df.mean(axis=1), 2) # Average sales value for the month periods
        row_average_df = pd.DataFrame({'Avg Sales value past 4 months': row_average}) # create dataframe of series object 'row_average'
        part_newdf = part_newdf.join(row_average_df) # add row_average df to the dataframe

        # Filter for stores based on thresholds
        threshold_value = 0.3
        part_newdf['Lower Threshold past 4 months'] = round(part_newdf['Avg Sales value past 4 months']* (1-threshold_value), 2)
        part_newdf['Upper Threshold past 4 months'] = round(part_newdf['Avg Sales value past 4 months']* (1+threshold_value), 2)

        # Filter and Flag
        # Create boolean masks for each condition
        within_limit = part_newdf['Sales value March'].between(part_newdf['Lower Threshold past 4 months'], part_newdf['Upper Threshold past 4 months'])
        upper_breach = part_newdf['Sales value March'] > part_newdf['Upper Threshold past 4 months']
        lower_breach = part_newdf['Sales value March'] < part_newdf['Lower Threshold past 4 months']

        # Define the corresponding flags for each condition
        conditions = [within_limit, upper_breach, lower_breach]
        choices = ['Within limit', 'Upper threshold breached', 'Lower threshold breached']

        # Use numpy.select to assign the corresponding flag to each row
        part_newdf['Sales Value Status'] = np.select(conditions, choices, default='Unknown')

        # Store values that breach limits
        full_df = part_newdf[~(part_newdf['Sales Value Status']=='Within limit')]

        return full_df

    

In [16]:
#Create an instance of the SalesAnomalyDetector class by passing the file path of the Excel file as an argument. For example:
detector = ItemAnomalyDetector(file_path=r'C:\Users\Akosua\Documents\Andrew\Interview Assessment Redo\DS Maverick Sample Dataset.xlsx')

#Call the detect_anomalies() method on the instance. This will return a DataFrame containing the stores with sales anomalies.
anomalies_df = detector.item_sales_anomaly_func()

#Print the DataFrame to see the results.
print(anomalies_df)

           Item Code                                          ITEM NAME  \
0     BAMBUT16377348  PRESIDENT  UNSALTED BUTTER   200G WRAPPED LACT...   
1      BAMBUT2251794  AMBASSADOR PRESIDENT  SALTED BUTTER   200G WRA...   
2     BAMBUT22957980    INCOLAC  SALTED BUTTER   200G WRAPPED BELGOMILK   
3     BAMBUT23427855  HERITAGE  GOURMET BUTTER   200G WRAPPED HERITA...   
5     BAMCHT11686857  TOP CHOCO ORIGINAL    18G SACHET ALPHA INDUSTR...   
...              ...                                                ...   
5013  TSOSLT91393400  DIVA ROSE   MOISTURIZING 125G SACHET INTERCONT...   
5014   TSOSLT9644992  GEISHA  ALOE VERA & HONEY BAR  100G WRAPPED UN...   
5015   TSOSLT9644992  GEISHA  ALOE VERA & HONEY BAR  100G WRAPPED UN...   
5016   TSOSLT9644992  GEISHA  ALOE VERA & HONEY BAR  100G WRAPPED UN...   
5017  TSOSTL56347125  DETTOL    MAXIMUM PROTECTION 160G BOX RECKITT ...   

           CATEGORY  Sales value November  Sales value December  \
0     EDIBLE SPREAD             

C:\Users\Akosua\AppData\Local\Temp\ipykernel_17664\2371292343.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Non-zero-sales'] = sales_df['SALES'].abs() # Compute non-zero sales
C:\Users\Akosua\AppData\Local\Temp\ipykernel_17664\2371292343.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Sales Value'] = sales_df['Non-zero-sales']* sales_df['PRICE'] # Compute sales value column


### Making Month selection for Sales Value computation scalable to different months

#### On Store Analysis

In [3]:
def sales_anomaly_func(file):
    """
    This function compares the sales value recorded by shops in the present month against the past
    4 months and determines whether the present sales value has defects
    """
    df = pd.read_excel(file) # Read Excel file
    
    sales_df = df[["Period", "Store Code", "Store Name", "Auditors Name", "SALES", "PRICE"]] # Subset relevant columns and create dataframe 

    sales_df['Non-zero-sales'] = sales_df['SALES'].abs() # Compute non-zero sales 

    sales_df['Sales Value'] = sales_df['Non-zero-sales']* sales_df['PRICE'] # Compute sales value column

    subset = sales_df.groupby(["Period", "Store Code", "Store Name", "Auditors Name"])['Sales Value'].sum().reset_index(name='Sales Value') # groupby

    # Arrange desired record in pivot table
    table = pd.pivot_table(subset, values= ['Sales Value'],
                           index= ["Store Code", "Store Name", "Auditors Name"],
                           columns= ['Period'])
    
    new_sales = table.reset_index()#.stack() # change shape of pivot table

    # Retrieve Month-on-Month sales values for each of the past 4 months and the current month
    df_list = []
    months_weeks = [16, 12, 8, 4, 0] 
    num = iter(list(range(4,new_sales.shape[1])))
    part_newdf = new_sales.iloc[:, :3]  # subset indexed pivot table to pick out Store Code, Store Name and Auditor's Name
    part_newdf.columns = part_newdf.columns.droplevel(level=1) # remove the period index  
    for week in months_weeks:
        td = dt.timedelta(weeks=week)
        month_sales = sales_df.iloc[-1, 0] - td
        sample_df = pd.DataFrame({f'Sales value {month_sales.month_name()}': new_sales.loc[:, new_sales.columns[next(num)]]})
        df_list.append(sample_df)
    concatenated_df = pd.concat(df_list, axis=1)
    part_newdf = part_newdf.join(concatenated_df)

    # Compute mean of Sales value per store for the past 4 months
    past_4_months_df = part_newdf.iloc[:, 3:7]  #  Selects the past 4 month sales averages
    row_average = round(past_4_months_df.mean(axis=1), 2) # Average sales value for the month periods
    row_average_df = pd.DataFrame({'Avg Sales value past 4 months': row_average}) # create dataframe of series object 'row_average'
    part_newdf = part_newdf.join(row_average_df) # add row_average df to the dataframe

    # Filter for stores based on thresholds
    threshold_value = 0.3
    part_newdf['Lower Threshold past 4 months'] = round(part_newdf['Avg Sales value past 4 months']* (1-threshold_value), 2)
    part_newdf['Upper Threshold past 4 months'] = round(part_newdf['Avg Sales value past 4 months']* (1+threshold_value), 2)

    # Filter and Flag
    # Create boolean masks for each condition
    within_limit = part_newdf['Sales value March'].between(part_newdf['Lower Threshold past 4 months'], part_newdf['Upper Threshold past 4 months'])
    upper_breach = part_newdf['Sales value March'] > part_newdf['Upper Threshold past 4 months']
    lower_breach = part_newdf['Sales value March'] < part_newdf['Lower Threshold past 4 months']

    # Define the corresponding flags for each condition
    conditions = [within_limit, upper_breach, lower_breach]
    choices = ['Within limit', 'Upper threshold breached', 'Lower threshold breached']

    # Use numpy.select to assign the corresponding flag to each row
    part_newdf['Sales Value Status'] = np.select(conditions, choices, default='Unknown')

    # Store values that breach limits
    full_df = part_newdf[~(part_newdf['Sales Value Status']=='Within limit')]

    # # Open the Excel file using ExcelWriter
    # with pd.ExcelWriter('programming-assessment.xlsx', engine='openpyxl', mode='a') as writer:
    #     # Add the DataFrame as a new worksheet
    #     full_df.to_excel(writer, sheet_name='sales-store-checker-new')

    return part_newdf[~(part_newdf['Sales Value Status']=='Within limit')]

sales_anomaly_func(r"C:\Users\Akosua\Documents\Andrew\Interview Assessment Redo\DS Maverick Sample Dataset.xlsx")

    

C:\Users\Akosua\AppData\Local\Temp\ipykernel_17208\1784539290.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Non-zero-sales'] = sales_df['SALES'].abs() # Compute non-zero sales
C:\Users\Akosua\AppData\Local\Temp\ipykernel_17208\1784539290.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Sales Value'] = sales_df['Non-zero-sales']* sales_df['PRICE'] # Compute sales value column


,Store Code,Store Name,Auditors Name,Sales value November,Sales value December,Sales value January,Sales value February,Sales value March,Avg Sales value past 4 months,Lower Threshold past 4 months,Upper Threshold past 4 months,Sales Value Status
1,GH000111433,NO NAME CHRISTY,LISA BOATENG,1591.90,1551.00,3207.70,3647.2,4976.46,2499.45,1749.61,3249.28,Upper threshold breached
3,GH000111456,JOLLO SHOPPING,LISA BOATENG,815.90,1517.70,2452.70,3592.4,4445.50,2094.67,1466.27,2723.07,Upper threshold breached
4,GH000111466,STELLA ENTERPRISE,LISA BOATENG,3945.30,2381.60,3580.30,3754.6,5159.20,3415.45,2390.81,4440.08,Upper threshold breached
5,GH000111474,EMMA ENTERPRISE,LISA BOATENG,1290.10,1894.60,3219.50,3261.9,4200.40,2416.52,1691.56,3141.48,Upper threshold breached
6,GH000111489,NO NAME DORIS,LISA BOATENG,543.90,624.30,789.00,1526.3,1503.70,870.88,609.62,1132.14,Upper threshold breached
...,...,...,...,...,...,...,...,...,...,...,...,...
199,GH000512745,CEE LOVELY SUPERMARKET,LEWIS AGBOW,1992.40,4289.95,6241.35,8026.2,11465.65,5137.48,3596.24,6678.72,Upper threshold breached
200,GH000512761,PSALM 21,LEWIS AGBOW,1062.60,855.90,1745.00,1980.3,1931.70,1410.95,987.66,1834.24,Upper threshold breached
202,GH0005127810,AMENUVEVE ENT,LEWIS AGBOW,1786.35,1972.80,2984.60,4072.6,6204.10,2704.09,1892.86,3515.32,Upper threshold breached
203,GH000512797,ADINKRA ENT.,LEWIS AGBOW,1005.70,1215.00,2261.75,1920.1,2263.25,1600.64,1120.45,2080.83,Upper threshold breached


##### On Item Analysis

In [33]:
def item_sales_anomaly_func(file):
    """
    This function compares the sales value recorded by shops in the present month against the past
    4 months and determines whether the present sales value has defects
    """
    df = pd.read_excel(file) # Read Excel file
    
    # Subset and create dataframe with relevant columns to sales analysis
    sales_df = df[["Period", "Item Code", "ITEM NAME", "CATEGORY", "Auditors Name", "SALES", "PRICE"]]

    sales_df['Non-zero-sales'] = sales_df['SALES'].abs() # Compute non-zero sales 

    sales_df['Sales Value'] = sales_df['Non-zero-sales']* sales_df['PRICE'] # Compute sales value column

    subset = sales_df.groupby(["Period", "Item Code", "ITEM NAME", "CATEGORY", "Auditors Name"])['Sales Value'].sum().reset_index(name='Sales Value')

    # Arrange desired record in pivot table
    table = pd.pivot_table(subset, values= ['Sales Value'],
                           index= ["Item Code", "ITEM NAME", "CATEGORY", "Auditors Name"],
                           columns= ['Period'])
    
    new_sales = table.reset_index()#.stack()

    # Retrieve Month-on-Month sales values for each of the past 4 months and the current month
    df_list = []
    months_weeks = [16, 12, 8, 4, 0]
    num = iter(list(range(4,new_sales.shape[1])))
    part_newdf = new_sales.iloc[:, :3]  # subset indexed pivot table to pick out Store Code, Store Name and Auditor's Name
    part_newdf.columns = part_newdf.columns.droplevel(level=1) # remove the period index  
    for week in months_weeks:
        td = dt.timedelta(weeks=week)
        month_sales = sales_df.iloc[-1, 0] - td
        sample_df = pd.DataFrame({f'Sales value {month_sales.month_name()}': new_sales.loc[:, new_sales.columns[next(num)]]})
        df_list.append(sample_df)
    concatenated_df = pd.concat(df_list, axis=1)
    part_newdf = part_newdf.join(concatenated_df)

    # Compute mean of Sales value per store for the past 4 months
    past_4_months_df = part_newdf.iloc[:, 4:8]  #  Selects the past 4 month sales averages
    row_average = round(past_4_months_df.mean(axis=1), 2) # Average sales value for the month periods
    row_average_df = pd.DataFrame({'Avg Sales value past 4 months': row_average}) # create dataframe of series object 'row_average'
    part_newdf = part_newdf.join(row_average_df) # add row_average df to the dataframe

    # Filter for stores based on thresholds
    threshold_value = 0.3
    part_newdf['Lower Threshold past 4 months'] = round(part_newdf['Avg Sales value past 4 months']* (1-threshold_value), 2)
    part_newdf['Upper Threshold past 4 months'] = round(part_newdf['Avg Sales value past 4 months']* (1+threshold_value), 2)

    # Filter and Flag
    # Create boolean masks for each condition
    within_limit = part_newdf['Sales value March'].between(part_newdf['Lower Threshold past 4 months'], part_newdf['Upper Threshold past 4 months'])
    upper_breach = part_newdf['Sales value March'] > part_newdf['Upper Threshold past 4 months']
    lower_breach = part_newdf['Sales value March'] < part_newdf['Lower Threshold past 4 months']

    # Define the corresponding flags for each condition
    conditions = [within_limit, upper_breach, lower_breach]
    choices = ['Within limit', 'Upper threshold breached', 'Lower threshold breached']

    # Use numpy.select to assign the corresponding flag to each row
    part_newdf['Sales Value Status'] = np.select(conditions, choices, default='Unknown')

    # Store values that breach limits
    full_df = part_newdf[~(part_newdf['Sales Value Status']=='Within limit')]

    # # Open the Excel file using ExcelWriter
    # with pd.ExcelWriter('programming-assessment.xlsx', engine='openpyxl', mode='a') as writer:
    #     # Add the DataFrame as a new worksheet
    #     full_df.to_excel(writer, sheet_name='sales-item-checker')

    return full_df#part_newdf[~(part_newdf['Sales Value Status']=='Within limit')]

item_sales_anomaly_func(r"C:\Users\Akosua\Documents\Andrew\Interview Assessment Redo\DS Maverick Sample Dataset.xlsx")

    